In [0]:
# Instala o pacote necessário para leitura de variáveis armazenadas em arquivo .env
%pip install python-dotenv

In [0]:
# Importa as bibliotecas
from dotenv import load_dotenv
import os

In [0]:
# Carrega as variáveis definidas no arquivo .env
load_dotenv()

In [0]:
# Teste simples (não imprime a chave real)
# if API_KEY:
#     print("✅ Chave carregada com sucesso!")
# else:
#     print("⚠️ Chave não encontrada. Verifique se o arquivo .env está no mesmo diretório.")

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze_etl
COMMENT 'Camada Bronze para armazenamento de dados brutos';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver_etl
COMMENT 'Camada Silver para armazenamento de dados tratados';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold_etl
COMMENT 'Camada Golf para armazenamento de dados prontos para uso';